In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'

In [2]:
#saga
data_dir = "/home/aeerik/data/raw/"


Data import

In [3]:
from data_preprocessing import data_loader
include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)
NCBI.head()

,year,location,genes
1,[PAD],USA,"[tet(A), sul1, aadA1]"
4,[PAD],USA,"[aph(6)-Id, sul2, tet(A), aph(3'')-Ib]"
5,[PAD],USA,"[glpT_E448K=POINT, pmrB_Y358N=POINT]"
6,[PAD],Sweden,"[glpT_E448K=POINT, uhpT_E350Q=POINT, cyaA_S352..."
7,1979,USA,[glpT_E448K=POINT]


Vocabulary

In [6]:
from build_vocabulary import make_vocabulary
vocabulary = make_vocabulary(NCBI)
print(len(vocabulary))

1208


In [3]:
from create_dataset import NCBIDataset
from build_vocabulary import make_vocabulary
from data_preprocessing import data_loader

include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)

max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

In [36]:
test_set[1][2]

tensor([[False,  True, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])

Embedding 

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class JointEmbedding(nn.Module):

    def __init__(self, embedding_dim, vocab_size, max_length, drop_prob):
        super(JointEmbedding, self).__init__()

        self.max_length = max_length
        self.drop_prob = drop_prob
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.token_emb = nn.Embedding(self.vocab_size, self.embedding_dim)

        self.dropout = nn.Dropout(drop_prob)	
        self.norm = nn.LayerNorm(self.embedding_dim)

    def forward(self, input_tensor):
        token_embedding = self.token_emb(input_tensor)
        token_embedding = self.norm(token_embedding)
        token_embedding = self.dropout(token_embedding)

        return token_embedding


In [5]:
from embedding import JointEmbedding
embedding_dim = 32
voca_size = len(vocabulary)
max_length = 20
drop_prob = 0.2

emb_test = JointEmbedding(embedding_dim, voca_size, max_length, drop_prob)
emb_test.forward(test_set[1][0])

tensor([[ 0.8331,  1.0515, -0.0000,  0.2314,  1.3775, -2.2667,  0.5048,  1.0534,
         -0.0000,  0.0000, -0.9512, -0.0132,  0.0000, -1.1990, -0.0790, -0.6929,
          1.5763,  1.6299, -1.0645, -0.7928, -1.1193, -0.2490,  0.3520,  0.2263,
          0.9369,  3.2176, -0.0000,  1.3471, -3.2385, -0.2307,  0.4862, -0.9063],
        [ 1.1834, -2.1201, -0.1374,  0.7954, -0.2633,  1.0959, -1.5578, -0.4190,
         -0.0000, -0.0855, -2.3608,  0.5345, -0.4668,  0.6931, -0.9211, -0.0000,
         -0.0000,  0.0000,  3.0867, -0.2822,  0.1557,  0.0000,  0.0966,  0.0080,
          0.0000, -1.0138,  0.5358,  0.4295,  0.0000,  1.3010,  0.0628, -0.0000],
        [-1.9641,  0.8865,  1.8891,  0.6134, -1.6141, -2.5473,  2.6708, -0.0801,
          0.0842,  0.0000, -0.8039, -1.1668,  0.2151, -1.6891,  1.9464,  1.2084,
          0.4367,  0.4825, -0.9501,  0.0000,  1.2927,  0.3222, -1.6566,  0.1560,
         -0.1427, -0.3243, -1.9109,  0.0000, -0.0779, -1.3170,  1.2141,  0.4513],
        [-1.1707,  1.2274

In [16]:
test_set[0][0]

tensor([  0,   1,  59, 213, 214, 215,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1])

In [5]:
import torch

from torch import nn
import torch.nn.functional as f

class AttentionHead(nn.Module):

    def __init__(self, dim_inp, dim_out, drop_prob):
        super(AttentionHead, self).__init__()

        self.dim_inp = dim_inp
        self.drop_prob = drop_prob
        self.dropout = nn.Dropout(drop_prob)
        self.q = nn.Linear(dim_inp, dim_out)
        self.k = nn.Linear(dim_inp, dim_out)
        self.v = nn.Linear(dim_inp, dim_out)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor = None):
        query, key, value = self.q(input_tensor), self.k(input_tensor), self.v(input_tensor)

        scale = query.size(1) ** 0.5
        scores = torch.matmul(query, key.transpose(-1, -2)) / scale

        scores = scores.masked_fill_(attention_mask, -1e9)
        attn = f.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, value)

        return context


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, dim_inp, dim_out,drop_prob):
        super(MultiHeadAttention, self).__init__()

        self.heads = nn.ModuleList([
            AttentionHead(dim_inp, dim_out,drop_prob) for _ in range(num_heads)
        ])
        self.linear = nn.Linear(dim_out * num_heads, dim_inp)
        self.norm = nn.LayerNorm(dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        s = [head(input_tensor, attention_mask) for head in self.heads]
        scores = torch.cat(s, dim=-1)
        scores = self.linear(scores)
        return self.norm(scores)
    
attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

multihead_test = MultiHeadAttention(8, 32, 32,0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-6.2529e-02,  6.6882e-01,  1.2920e+00,  1.0694e+00, -7.1454e-02,
          6.7509e-01, -2.0224e-02, -1.4736e+00, -2.0233e+00,  2.4986e-01,
          2.4865e-01, -6.7741e-01,  9.8667e-01, -1.6409e+00, -9.8899e-01,
          9.9393e-01,  2.1556e-01,  1.6299e+00, -7.0088e-01,  1.1783e+00,
         -6.0879e-01, -1.0770e+00,  4.7027e-01,  1.0028e+00, -2.2437e-02,
         -6.6476e-01,  1.0250e-01, -2.1046e+00,  5.9789e-01,  1.6473e+00,
         -6.6898e-01, -2.2295e-01],
        [-1.4151e-01,  1.1083e+00,  9.1247e-01,  1.3594e+00, -4.5214e-01,
          5.3718e-01,  1.4342e-01, -9.5685e-01, -1.1647e+00,  5.0426e-01,
          1.0839e+00, -1.4875e-01,  7.5383e-01, -1.1176e+00, -6.3895e-01,
         -1.0583e-02,  7.7586e-01,  2.3473e+00, -8.6620e-01,  7.3578e-01,
         -6.8109e-01, -1.5421e+00,  2.6042e-02,  7.8927e-01, -6.3132e-01,
         -1.3392e+00, -1.5855e-03, -2.4988e+00,  3.9480e-01,  1.3509e+00,
         -5.6054e-01, -7.0681e-02],
        [-2.8954e-01,  2.0942e-01,  1.38

In [6]:
import torch

from torch import nn
import torch.nn.functional as f

#dropout 
#num heads
#dim in, dim out

class Encoder(nn.Module):

    def __init__(self, dim_inp, dim_out, attention_heads, dropout_prob):
        super(Encoder, self).__init__()
        self.dropout_prob = dropout_prob
        self.attention_heads = attention_heads
        self.dim_inp = dim_inp
        self.dim_out = dim_out
        
        
        self.attention = MultiHeadAttention(self.attention_heads, self.dim_inp, self.dim_out, self.dropout_prob)  # batch_size x sentence size x dim_inp
        self.feed_forward = nn.Sequential(
            nn.Linear(self.dim_inp, self.dim_out),
            nn.Dropout(self.dropout_prob),
            nn.GELU(),
            nn.Linear(self.dim_out, self.dim_inp),
            nn.Dropout(self.dropout_prob)
        )
        self.norm = nn.LayerNorm(self.dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        context = self.attention(input_tensor, attention_mask)
        res = self.feed_forward(context)
        return self.norm(res)

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[ 4.1706e-01, -6.7473e-02,  5.0605e-01, -9.7665e-02,  8.3774e-01,
         -6.7473e-02, -1.4544e+00, -3.3227e-01,  1.5943e+00,  1.1752e+00,
         -6.7473e-02,  1.2708e-01, -1.1516e+00, -6.7473e-02, -6.7473e-02,
          7.5421e-01,  9.5889e-01, -8.1097e-01, -6.7473e-02, -6.7473e-02,
          1.4775e+00, -3.3237e+00, -1.0164e+00, -6.7473e-02, -6.6237e-01,
          7.5044e-01, -6.7473e-02, -3.7708e-01,  2.1856e+00, -9.8690e-01,
         -6.7473e-02,  1.0412e-01],
        [ 3.5822e-01, -7.5941e-02,  4.7088e-02,  7.7096e-02,  1.6481e+00,
          1.6597e+00, -3.4294e-01, -7.8363e-01,  1.2021e+00,  4.4501e-01,
          1.2442e-01,  5.1303e-02,  7.0817e-01, -1.8316e+00,  9.5101e-01,
          7.7096e-02,  9.8989e-01,  5.2216e-01,  8.0485e-01, -1.4087e+00,
          1.1965e+00, -2.3475e+00, -8.9523e-01, -1.2553e+00, -1.0449e+00,
         -7.6363e-01,  7.7096e-02, -1.0664e+00,  1.5044e+00,  7.7096e-02,
         -9.3702e-01,  2.3154e-01],
        [ 6.9456e-01,  3.4231e-01, -1.49

In [7]:
class BERT(nn.Module):

    def __init__(self, vocab_size, max_length, dim_inp, dim_out, attention_heads, num_encoders, dropout_prob):
        super(BERT, self).__init__()
        self.attention_heads = attention_heads
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.dim_inp = dim_inp  
        self.dim_out = dim_out
        self.num_encoders = num_encoders
        self.dropout_prob = dropout_prob  

        self.embedding = JointEmbedding(self.dim_inp, self.vocab_size, self.max_length, self.dropout_prob)
        self.encoders = nn.ModuleList([Encoder(self.dim_inp, self.dim_out, self.attention_heads, self.dropout_prob) for _ in range(self.num_encoders)])

        self.token_prediction_layer = nn.Linear(self.dim_inp, self.vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        embedded = self.embedding(input_tensor)
        for layer in self.encoders:
            embedded = layer(embedded, attention_mask)

        token_predictions = self.token_prediction_layer(embedded)
        return self.softmax(token_predictions)

In [6]:
bert_test = BERT(len(vocabulary), 20, 32, 32, 8, 2, 0.2)
bert_test.forward(test_set[1][0], test_set[1][2])

NameError: name 'BERT' is not defined

In [7]:
from bert_builder import AttentionHead

attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-3.8662e-01, -8.5011e-02,  2.2547e-01, -1.6851e-02,  4.4351e-02,
          1.4062e-02,  1.0866e-02,  1.5393e-01, -4.3348e-01,  2.5300e-01,
          1.9350e-01, -3.6733e-01, -5.4809e-02, -2.0595e-01,  1.3405e-01,
          1.4189e-01,  6.8620e-02,  1.6805e-01,  4.8116e-01,  4.3453e-01,
          3.9839e-01, -5.9823e-01, -3.6819e-01, -2.1596e-01,  8.2339e-02,
          2.0398e-01,  1.7603e-01,  7.2742e-01, -1.4394e-03, -2.7836e-01,
          3.4622e-01,  2.9143e-02],
        [-1.1216e-01, -4.0262e-01,  1.4410e-01, -1.4561e-01, -2.6659e-02,
          6.7763e-02, -1.2143e-01,  1.7912e-01, -1.8841e-01,  1.3643e-01,
          2.9305e-01, -2.1795e-01, -1.3427e-01, -1.5704e-01, -5.3516e-03,
          1.5155e-01, -2.9202e-01,  2.3277e-01,  2.0884e-01,  4.8633e-01,
          2.1722e-01, -5.5967e-01, -8.6852e-02,  1.4894e-02,  9.6428e-02,
          1.9595e-01,  1.4897e-01,  4.3382e-01,  6.8461e-02, -2.5231e-01,
          2.1963e-01,  2.1233e-01],
        [-3.9743e-01, -2.5733e-01,  1.38

In [8]:
from bert_builder import MultiHeadAttention
multihead_test = MultiHeadAttention(num_heads=8, dim_inp=32, dim_out=32, drop_prob=0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[ 1.1050, -0.6418, -1.4463, -0.0554,  0.3496,  1.2113,  2.5580, -1.1008,
         -1.0964, -0.2305,  1.7568,  1.1890, -1.1085, -1.2818, -0.3666, -1.2891,
          1.2389,  0.8882, -0.7111, -1.1318, -0.2111,  0.3702, -0.1092, -0.9846,
          0.0066,  0.4778,  0.8445, -0.6239,  0.1886, -0.9714,  0.2523,  0.9230],
        [ 0.9288, -1.2360, -1.2010, -0.6093, -0.1523,  0.6816,  2.7068, -1.1619,
         -0.7128,  0.2009,  1.7770,  0.8195, -0.3951, -0.9982, -0.0635, -1.2012,
          1.0870,  0.8328, -1.1834, -1.5427, -0.3221,  0.5871, -0.2614, -1.6084,
         -0.1981,  0.4088,  0.6683, -0.1010,  0.4177, -0.0643,  0.5961,  1.3005],
        [ 1.0855, -0.4137, -1.7830,  0.1882,  0.5005,  0.3475,  2.9796, -1.2225,
         -0.0425,  0.4861,  1.4213,  0.8477, -0.4339, -1.8019, -0.1404, -1.7669,
          1.3558, -0.9298, -0.0692, -0.6686, -0.2882, -0.0353,  0.1281, -1.0005,
          0.0204,  0.6411,  0.6273, -0.1339, -0.0829, -0.8591, -0.0956,  1.1387],
        [ 1.7917,  0.1199

In [9]:
from bert_builder import Encoder

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-2.3107,  0.0484,  0.3815,  0.0484, -0.3970, -1.0474,  0.7867, -0.5924,
         -1.7959,  0.8987,  1.7071, -0.3569, -0.5909,  0.3377,  0.0484,  0.0484,
         -0.8283,  0.3655,  0.0484, -1.9181,  0.9739,  0.6465,  2.1926,  0.7340,
         -0.3556,  0.0484,  0.0484, -0.2997, -1.1932,  0.8504,  1.7545, -0.2819],
        [-0.1738,  2.4849,  2.1562, -1.1376, -0.7600, -0.3139,  1.0181, -0.2832,
          0.8820,  0.4721, -1.4676, -1.1608, -0.1738, -2.3181, -0.5207,  1.1506,
         -0.1738, -0.1738, -0.1738, -1.0155,  0.3465,  1.3585,  0.2631,  1.3317,
         -0.8844, -0.7417, -0.1738, -0.0424, -0.1738,  0.1367,  0.4259, -0.1642],
        [-2.5320,  1.8605,  1.1375,  0.5034,  0.1041, -0.7443,  0.7407, -0.0237,
          0.1041,  0.9539,  0.1041,  0.2748, -0.7883,  0.7739,  0.1041, -0.0399,
         -1.8362,  0.1041,  0.1041, -1.9907,  0.8502, -0.7054,  2.1488,  0.6838,
         -1.1042,  0.6828,  0.1041, -0.4223,  0.1041, -0.6314,  0.4228, -1.0476],
        [-2.2955,  2.1124

In [11]:
from bert_builder import BERT

bert_test = BERT(vocab_size=len(vocabulary), max_length=20, dim_inp=32, dim_out=32, attention_heads=8, num_encoders=2, dropout_prob=0.2)
bert_test.forward(test_set[1][0], test_set[1][2])

tensor([[-7.3623, -7.1583, -7.7241,  ..., -7.4518, -8.0045, -6.4266],
        [-7.1923, -7.0146, -7.9684,  ..., -6.5024, -7.6257, -6.6102],
        [-7.8528, -8.0000, -7.1180,  ..., -6.2702, -7.6824, -7.6966],
        ...,
        [-7.7168, -7.1282, -7.0677,  ..., -6.5226, -7.2996, -7.2195],
        [-7.3509, -8.0282, -6.5848,  ..., -6.2151, -7.2776, -7.3208],
        [-7.4697, -7.6191, -7.3346,  ..., -6.4897, -7.3715, -6.8816]],
       grad_fn=<LogSoftmaxBackward0>)

---------------------------

dataset
- behöver vocabulary
- saknar: getitem och prepare dataset som kallar på construct_masking

In [1]:
from torchtext.vocab import vocab 
from copy import deepcopy
from collections import Counter
import torch
from torch.utils.data import Dataset
import pandas as pd
from itertools import chain
import numpy as np

from data_preprocessing import data_loader

include_pheno = True
threshold_year = 1970

NCBI = data_loader(include_pheno,threshold_year)
NCBI.fillna('[PAD]', inplace=True)
print('data importerad')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MASK_PERCENTAGE = 0.15


# data, vocabulary, max sequence length, mask probability, include sequences, some random state
class NCBIDataset(Dataset):

    MASKED_INDICES_COLUMN = 'masked_indices'
    TARGET_COLUMN = 'indices'
    NSP_TARGET_COLUMN = 'is_next'
    TOKEN_MASK_COLUMN = 'token_mask'

    def __init__(self,
                 data: pd.DataFrame,
                 vocab: vocab,
                 max_seq_len: int,
                 mask_prob: float,
                 random_state: int = 23,
                 ):
        
        self.random_state = random_state
        np.random.seed(self.random_state)

        CLS = '[CLS]'
        PAD = '[PAD]'
        MASK = '[MASK]'
        UNK = '[UNK]'

        self.data = data.reset_index(drop=True) 
        self.num_samples = self.data.shape[0]
        self.vocab = vocab
        self.vocab_size = len(self.vocab)
        self.CLS = CLS 
        self.PAD = PAD
        self.MASK = MASK
        self.UNK = UNK
        self.max_seq_len = max_seq_len
        self.mask_prob = mask_prob
        self.columns = [self.MASKED_INDICES_COLUMN, self.TARGET_COLUMN]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input = torch.Tensor(item[self.MASKED_INDICES_COLUMN],device=device).long()
        token_mask  = torch.tensor(item[self.TARGET_COLUMN], device=device).long()
        attention_mask = (input == self.vocab[self.PAD]).unsqueeze(0)

        return input, token_mask , attention_mask

    def _construct_masking(self):
        sequences = deepcopy(self.data['genes'].tolist())
        masked_sequences = list()
        target_indices_list = list()
        seq_starts = [[self.CLS, self.data['year'].iloc[i], self.data['location'].iloc[i]] for i in range(self.data.shape[0])]

        for i, geno_seq in enumerate(sequences):
            seq_len = len(geno_seq)
            masking_index = np.random.rand(seq_len) < self.mask_prob   
            target_indices = np.array([-1]*seq_len)
            indices = masking_index.nonzero()[0]
            target_indices[indices] = self.vocab.lookup_indices([geno_seq[i] for i in indices])
            for i in indices:
                r = np.random.rand()
                if r < 0.8:
                    geno_seq[i] = self.MASK
                elif r > 0.9:
                    geno_seq[i] = self.vocab.lookup_token(np.random.randint(self.vocab_size))
            geno_seq = seq_starts[i] + geno_seq
            target_indices = [-1]*3 + target_indices.tolist() 
            masked_sequences.append(geno_seq)
            target_indices_list.append(target_indices)
        print('här är lugnt')
        print(masked_sequences[:20])
        print(target_indices_list[:20])
        masked_sequences = [seq + [self.PAD]*(self.max_seq_len - len(seq)) for seq in masked_sequences]
        print(masked_sequences[:20])
        print(range(len(target_indices_list)))
        for i in range(len(target_indices_list)):
            indices = target_indices_list[i]
            padding = [-1] * (self.max_seq_len - len(indices))
            target_indices_list[i] = indices + padding
        print('hit men inte längre ')
        return masked_sequences, target_indices_list 
        
    def prepare_dataset(self):
        masked_sequences, target_indices = self._construct_masking()
        indices_masked = [self.vocab.lookup_indices(masked_seq) for masked_seq in masked_sequences]

        rows = zip(indices_masked, target_indices)
        self.data = pd.DataFrame(rows, columns=self.columns)
        print(self.data.head())

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, UNK])
    return vocabulary


max_length = 10
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

data importerad
här är lugnt
[['[CLS]', '[PAD]', 'USA', 'aadA1', 'sul1', 'tet(A)'], ['[CLS]', '[PAD]', 'Sweden', '[MASK]', '[MASK]', 'tet(A)', '[MASK]'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'glpT_E448K=POINT', 'cyaA_S352T=POINT', 'uhpT_E350Q=POINT'], ['[CLS]', '1979', 'USA', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', '[MASK]', '[MASK]'], ['[CLS]', '[PAD]', 'USA', '[MASK]', 'pmrB_E123D=POINT', '[MASK]'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'parC_A56T=POINT', 'glpT_E448K=POINT', '[MASK]', '[MASK]'], ['[CLS]', '[PAD]', 'Sweden', 'parE_I355T=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'Sweden', "aph(3'')-Ib", 'tet(B)', 'sul2', '[MASK]'], ['[CLS]', '[PAD]', 'Indonesia', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_E123D=PO

In [2]:
from torchtext.vocab import vocab 
from copy import deepcopy
from collections import Counter
import torch
from torch.utils.data import Dataset
import pandas as pd
from itertools import chain
import numpy as np

from data_preprocessing import data_loader
from create_dataset import NCBIDataset

include_pheno = True
threshold_year = 1970

NCBI = data_loader(include_pheno,threshold_year)
print('data importerad')

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, UNK])
    return vocabulary


max_length = 10
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

data importerad
här är lugnt
[['[CLS]', nan, 'USA', 'sul1', 'tet(A)', 'aadA1'], ['[CLS]', nan, 'Sweden', '[MASK]', '[MASK]', 'aph(6)-Id', '[MASK]'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'uhpT_E350Q=POINT', 'cyaA_S352T=POINT', 'glpT_E448K=POINT'], ['[CLS]', '1979', 'USA', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_E123D=POINT', '[MASK]', '[MASK]'], ['[CLS]', nan, 'USA', '[MASK]', 'pmrB_E123D=POINT', '[MASK]'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'pmrB_Y358N=POINT', 'tet(A)', '[MASK]', '[MASK]'], ['[CLS]', nan, 'Sweden', 'parE_I355T=POINT'], ['[CLS]', nan, 'Sweden', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'sul2', "aph(3'')-Ib", 'aph(6)-Id', '[MASK]'], ['[CLS]', nan, 'Indonesia', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_E123D=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'aph(6)-Id', 'glpT_E448K=POINT

: 

Vocabulary

In [ ]:
import random
import typing
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from itertools import chain
import torch

from tqdm import tqdm
from torch.utils.data import Dataset
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer 

print(list(chain(*NCBI['genes']))[:20])

mylist = list(dict.fromkeys(list(chain(*NCBI['genes']))))
print(len(mylist))
print(mylist[:20])

test = list(chain(list(NCBI['year'])))
print(test[:40])
test = list(dict.fromkeys(list(chain(list(NCBI['year'])))))
print(len(test))
print(test[:40])

test = list(chain(list(NCBI['location'])))
print(test[:20])
test = list(dict.fromkeys(list(chain(list(NCBI['location'])))))
print(len(test))
print(test[:20])

In [ ]:
import random
import typing
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from itertools import chain
import torch

from tqdm import tqdm
from torch.utils.data import Dataset
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer 

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    SEP = '[SEP]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    print(token_list)
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, SEP, UNK])
    print(vocabulary)
    return vocabulary
vocabulary = make_vocabulary(NCBI)


Counter({'nan': 2, '1979': 1, '2013': 1, '1996': 1, '1975': 1, '2009': 1, '1974': 1, '1998': 1, '1983': 1, '2008': 1, '1995': 1, '1989': 1, '2003': 1, '1997': 1, '1994': 1, '2001': 1, '1980': 1, '1970': 1, '1971': 1, '2007': 1, '2004': 1, '2006': 1, '1982': 1, '1991': 1, '1990': 1, '2002': 1, '1988': 1, '1987': 1, '2011': 1, '1986': 1, '1985': 1, '2010': 1, '2005': 1, '2012': 1, '1984': 1, '2000': 1, '2014': 1, '2015': 1, '1999': 1, '1992': 1, '1973': 1, '1981': 1, '1993': 1, '2016': 1, '1972': 1, '1976': 1, '1977': 1, '2017': 1, '1978': 1, '2018': 1, '2019': 1, '2020': 1, '2021': 1, '2022': 1, '2023': 1, '2024': 1, 'USA': 1, 'Sweden': 1, 'Indonesia': 1, 'Canada': 1, 'Papua New Guinea': 1, 'Japan': 1, 'China': 1, 'Germany': 1, 'India': 1, 'Bangladesh': 1, 'Brazil': 1, 'UK': 1, 'Denmark': 1, 'South Korea': 1, 'France': 1, 'Norway': 1, 'Malaysia': 1, 'Slovenia': 1, 'Thailand': 1, 'Italy': 1, 'Israel': 1, 'Austria': 1, 'Belgium': 1, 'Guinea-Bissau': 1, 'Australia': 1, 'Hungary': 1, 'Leban

In [ ]:
vocabulary = make_vocabulary(NCBI)
